In [1]:
import polars as pl
from pathlib import Path


dataset_path = Path("../dataset/")
unnecessary_columns = ["posting_date", "id"]

train_df = pl.read_csv(dataset_path / "projectA_vehicle_train.csv").drop(
    unnecessary_columns
)
val_df = pl.read_csv(dataset_path / "projectA_vehicle_val.csv").drop(
    unnecessary_columns
)
test_df = pl.read_csv(dataset_path / "projectA_vehicle_test.csv").drop(
    unnecessary_columns
)

In [3]:
import optuna
import lightgbm as lgb
from src.metrics import rmse
from src.features.preprocess import Preprocessor
from src.suggest_params.preprocess import suggest_preprocessor_config
from src.suggest_params.regression import suggest_lgb_params


def objective(trial: optuna.Trial) -> float:
    # Optunaのトライアルからパラメータを取得
    lgb_params = suggest_lgb_params(trial)
    preprocessor_config = suggest_preprocessor_config(trial, task="regression")

    # 前処理
    preprocessor = Preprocessor(**preprocessor_config.to_dict())
    train_df_preprocessed, val_df_preprocessed, test_df_preprocessed = preprocessor.run(
        train_df, val_df, test_df
    )

    # LightGBMモデルの学習と評価
    train_set = lgb.Dataset(
        train_df_preprocessed.drop("price").to_pandas(),
        train_df_preprocessed["price"].to_pandas(),
    )
    val_set = lgb.Dataset(
        val_df_preprocessed.drop("price").to_pandas(),
        val_df_preprocessed["price"].to_pandas(),
        reference=train_set,
    )
    model = lgb.train(
        lgb_params,
        train_set,
        num_boost_round=1000,
        valid_sets=[val_set],
    )

    # バリデーションセットでの予測と評価
    val_pred = model.predict(val_df_preprocessed.drop("price").to_pandas())
    val_rmse = rmse(val_df_preprocessed["price"].to_numpy(), val_pred)

    return val_rmse


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)


[I 2025-07-23 16:35:53,219] A new study created in memory with name: no-name-f289a72b-3f2f-4c3a-85a7-09697023b91c
[I 2025-07-23 16:35:53,809] Trial 0 finished with value: 14374.43595598911 and parameters: {'num_leaves': 70, 'max_depth': 41, 'learning_rate': 0.01661987141459147, 'n_estimators': 238, 'min_child_samples': 55, 'feature_fraction': 0.5436665704259037, 'bagging_fraction': 0.41130499870939347, 'bagging_freq': 1, 'price_upper_bound': 14000, 'price_lower_bound': 2700, 'smoothing': 0.3244754926727199, 'min_samples_leaf': 5, 'noise_level': 0.27746037455688183}. Best is trial 0 with value: 14374.43595598911.


Validation RMSE: 14374.43595598911


[I 2025-07-23 16:35:54,501] Trial 1 finished with value: 13554.341811075888 and parameters: {'num_leaves': 64, 'max_depth': 6, 'learning_rate': 0.08131995083601078, 'n_estimators': 415, 'min_child_samples': 68, 'feature_fraction': 0.9411776599385393, 'bagging_fraction': 0.8271366180011662, 'bagging_freq': 4, 'price_upper_bound': 20500, 'price_lower_bound': 0, 'smoothing': 0.022635451528838866, 'min_samples_leaf': 10, 'noise_level': 0.05520489769391429}. Best is trial 1 with value: 13554.341811075888.


Validation RMSE: 13554.341811075888


[I 2025-07-23 16:35:56,048] Trial 2 finished with value: 9192.044425573466 and parameters: {'num_leaves': 69, 'max_depth': 25, 'learning_rate': 0.16660648553856336, 'n_estimators': 224, 'min_child_samples': 28, 'feature_fraction': 0.5666455087582927, 'bagging_fraction': 0.8853452214340901, 'bagging_freq': 2, 'price_upper_bound': 57000, 'price_lower_bound': 3000, 'smoothing': 0.5818629847235542, 'min_samples_leaf': 3, 'noise_level': 0.11607810273622834}. Best is trial 2 with value: 9192.044425573466.


...

Validation RMSE: 8906.527537139244


[I 2025-07-23 17:03:02,989] Trial 498 finished with value: 9099.659002945682 and parameters: {'num_leaves': 58, 'max_depth': 22, 'learning_rate': 0.01392239112350198, 'n_estimators': 442, 'min_child_samples': 11, 'feature_fraction': 0.980895042864427, 'bagging_fraction': 0.7624422780987131, 'bagging_freq': 7, 'price_upper_bound': 60000, 'price_lower_bound': 200, 'smoothing': 0.04672553784675922, 'min_samples_leaf': 2, 'noise_level': 0.08634588294897526}. Best is trial 470 with value: 8750.516607635713.


Validation RMSE: 9099.659002945682


[I 2025-07-23 17:03:06,859] Trial 499 finished with value: 9137.13275392966 and parameters: {'num_leaves': 61, 'max_depth': 18, 'learning_rate': 0.015031460305488055, 'n_estimators': 460, 'min_child_samples': 48, 'feature_fraction': 0.991259994017173, 'bagging_fraction': 0.7473729478282805, 'bagging_freq': 7, 'price_upper_bound': 59000, 'price_lower_bound': 100, 'smoothing': 0.034304111441550295, 'min_samples_leaf': 2, 'noise_level': 0.0909854656850294}. Best is trial 470 with value: 8750.516607635713.


Validation RMSE: 9137.13275392966


In [16]:
# save best preprocessor config as yaml
from src.config.preprocess import TargetEncoderConfig, PreprocessorConfig
from src.suggest_params.preprocess import (
    create_condition_config,
    create_cylinder_config,
    create_fuel_config,
    create_transmission_config,
    create_drive_config,
    create_manufacturer_config,
    create_paint_color_config,
    create_state_config,
    create_type_config,
)


def save_preprocessor_config(params: dict, output_path: Path) -> None:
    output_path.mkdir(parents=True, exist_ok=True)

    best_target_encoder_config = TargetEncoderConfig(
        smoothing=params["smoothing"],
        min_samples_leaf=params["min_samples_leaf"],
        noise_level=params["noise_level"],
    )

    # 各種エンコーダーの設定を作成
    condition_encoder_config = create_condition_config(best_target_encoder_config)
    cylinder_encoder_config = create_cylinder_config(best_target_encoder_config)
    fuel_encoder_config = create_fuel_config(best_target_encoder_config)
    transmission_encoder_config = create_transmission_config(best_target_encoder_config)
    drive_encoder_config = create_drive_config(best_target_encoder_config)
    manufacturer_encoder_config = create_manufacturer_config(best_target_encoder_config)
    paint_color_encoder_config = create_paint_color_config(best_target_encoder_config)
    state_encoder_config = create_state_config(best_target_encoder_config)
    type_encoder_config = create_type_config(best_target_encoder_config)

    # 最適な前処理設定を作成
    best_preprocessor_config = PreprocessorConfig(
        condition_encoder_config=condition_encoder_config,
        cylinder_encoder_config=cylinder_encoder_config,
        fuel_encoder_config=fuel_encoder_config,
        transmission_encoder_config=transmission_encoder_config,
        drive_encoder_config=drive_encoder_config,
        manufacturer_encoder_config=manufacturer_encoder_config,
        paint_color_encoder_config=paint_color_encoder_config,
        state_encoder_config=state_encoder_config,
        type_encoder_config=type_encoder_config,
        price_lower_bound=params["price_lower_bound"],
        price_upper_bound=params["price_upper_bound"],
    )

    # 保存
    best_preprocessor_config.to_yaml(output_path / "best_preprocessor_config_reg.yaml")


def save_lgb_params(params: dict, output_path: Path) -> None:
    output_path.mkdir(parents=True, exist_ok=True)
    best_lgb_params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": params["num_leaves"],
        "max_depth": params["max_depth"],
        "learning_rate": params["learning_rate"],
        "n_estimators": params["n_estimators"],
        "min_child_samples": params["min_child_samples"],
        "feature_fraction": params["feature_fraction"],
        "bagging_fraction": params["bagging_fraction"],
        "bagging_freq": params["bagging_freq"],
    }

    with open(output_path / "best_lgb_params_reg.yaml", "w") as f:
        for key, value in best_lgb_params.items():
            f.write(f"{key}: {value}\n")
    print(f"Best parameters saved to {output_path}")


save_preprocessor_config(study.best_params, Path("../params/"))
save_lgb_params(study.best_params, Path("../params/"))

Best parameters saved to ../params
